In [ ]:
# Problem 1 import libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
# Prob 1 explore data

customer_churn = pd.read_csv('Telco-Customer-Churn.csv')
print(customer_churn.head())
# Display summary statistics
print(customer_churn.describe())

In [ ]:
# Prob 1 Clean data
def load_data():
    # Load the data
    telco = pd.read_csv("Telco-Customer-Churn.csv")
    
    # Drop unnecessary columns
    telco.drop(columns=['customerID', 'PaymentMethod'], inplace=True)

    # Filter rows
    telco = telco[(telco['InternetService'] != "No") & (telco['PhoneService'] != "No")]

    # Reset index to clean up levels after filtering
    telco.reset_index(drop=True, inplace=True)

    # Drop additional columns
    telco.drop(columns=['PhoneService', 'TotalCharges'], inplace=True)

    telco['SeniorCitizen'] = telco['SeniorCitizen'].apply(lambda x: 'True' if x == 1 else 'False')

    # Convert object columns to categorical
    for col in telco.select_dtypes(include='object').columns:
        telco[col] = telco[col].astype('category')

    # Perform one-hot encoding, dropping the first category in each variable
    telco_encoded = pd.get_dummies(telco, drop_first=True)

    return telco_encoded

# Load the data
telco = load_data()

# Display the first few rows of the dataset
print(telco.head())

# Display summary statistics
print(telco.describe())

In [ ]:
# Prob 1a
# Filter rows where 'Churn' is 'Yes'
Churned = telco[telco['Churn_Yes'] == True]

# Calculate the average monthly charges for the resulting 'Churned' DataFrame
average_monthly_charges = Churned['MonthlyCharges'].mean()

# Print the average monthly charges
print(f"Average Monthly Charges for Churned Customers: ${round(average_monthly_charges,2)}")

# Assume a 12.5% profit margin
profit_margin = 0.125

# Calculate the expected cost of losing one customer (Cost 1)
cost_1 = average_monthly_charges * profit_margin

# Print the expected cost of losing one customer
print(f"Expected Cost of Losing One Customer (Cost 1): ${round(cost_1,2)}")


In [ ]:
#prob 1b
current_customer = telco[telco['Churn_Yes'] == False]

# Calculate the average monthly charges for the resulting 'current customers' DataFrame
current_customer_average_monthly_charges = current_customer['MonthlyCharges'].mean()

# Print the average monthly charges
print(f"Average Monthly Charges for current Customers: ${round(current_customer_average_monthly_charges,2)}")

# Assume a 12.5% profit margin
profit_margin_retained_customers = 0.075

# Calculate the expected cost of losing one customer (Cost 1)
cost_2 = current_customer_average_monthly_charges * profit_margin_retained_customers

# Print the expected cost of losing one customer
print(f"Expected Cost of Losing One Customer (Cost 2): ${round(cost_2,2)}")

In [ ]:
# Prob 1c

# Convert boolean columns to integers (0 and 1)
bool_columns = telco.select_dtypes(include=['bool']).columns
telco[bool_columns] = telco[bool_columns].astype(int)

# Define predictor variables and response variable
X = telco.drop(columns=['Churn_Yes'])  # Predictor variables (all columns except 'Churn')
y = telco['Churn_Yes']  # Response variable

# Add a constant to the predictor variables (intercept term)
X_with_intercept = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.Logit(y, X_with_intercept).fit()

# Print the summary of the model
print(logit_model.summary())

ypred = (logit_model.predict(X_with_intercept) > 0.5)

# Calculate false negatives (FN) and false positives (FP)
FN = ((y == True) & (ypred == False)).sum()
FP = ((y == False) & (ypred == True)).sum()

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y, ypred)

# Create a DataFrame for better visualization
conf_matrix_df = pd.DataFrame(conf_matrix, index=['True Neg-', 'True Pos+'], columns=['Pred Neg-', 'Pred Pos+'])

# Print the confusion matrix
print(conf_matrix_df)

# Loss=Cost 1∗FN+Cost 2∗FP
# I disagree with Cost 2. It should be 5%, since that is the lost revenue due to the unecessary discount. 7.5% is the profit margin of the retain customer.

In [ ]:
# Prob 1d
def calculate_loss(y_pred_i, y):
    FN = ((y == True) & (y_pred_i == False)).sum()
    FP = ((y == False) & (y_pred_i == True)).sum()
    return cost_1*FN + cost_2*FP

# Calculate loss for 1000 evenly distributed thresholds
thresholds = np.arange(0.0, 1.0, .001)
losses = []

for threshold in thresholds:
    y_pred_i = (logit_model.predict(X_with_intercept) > threshold)
    loss = calculate_loss(y_pred_i, y)
    losses.append(loss)

print(losses)
# Find the optimal threshold
optimal_threshold = thresholds[np.argmin(losses)]
print(f"Optimal Threshold: {optimal_threshold}")

# Plot the loss with respect to the threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds, losses, marker='o', markersize=2)
plt.xlabel('Threshold')
plt.ylabel('Loss (FN + FP)')
plt.title('Loss with Respect to Threshold')
plt.grid(True)
plt.show()

In [ ]:
# Problem 4 import libaries
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression

In [191]:
# Prob 4 explore data

letters = pd.read_csv('letters-2.csv')
print(letters.dtypes)
print(letters.head())

# Display summary statistics
print(letters.describe())

lettr    object
x.box     int64
y.box     int64
width     int64
high      int64
onpix     int64
x.bar     int64
y.bar     int64
x2bar     int64
y2bar     int64
xybar     int64
x2ybr     int64
xy2br     int64
x.ege     int64
xegvy     int64
y.ege     int64
yegvx     int64
dtype: object
  lettr  x.box  y.box  width  high  onpix  x.bar  y.bar  x2bar  y2bar  xybar  \
0     I      5     12      3     7      2     10      5      5      4     13   
1     D      4     11      6     8      6     10      6      2      6     10   
2     N      7     11      6     6      3      5      9      4      6      4   
3     G      2      1      3     1      1      8      6      6      6      6   
4     S      4     11      5     8      3      8      8      6      9      5   

   x2ybr  xy2br  x.ege  xegvy  y.ege  yegvx  
0      3      9      2      8      4     10  
1      3      7      3      7      3      9  
2      4     10      6     10      2      8  
3      5      9      1      7      5     10  
4  

In [192]:
# 4 Prepar X and Y
# Split data
letters_shuffled = letters.sample(frac=1, random_state=42).reset_index(drop=True)
train_size = int(0.75 * len(letters_shuffled))
train_set = letters_shuffled[:train_size]
test_set = letters_shuffled[train_size:]

# Display the sizes of the training and testing sets
print(f"Training set size: {train_set.shape[0]}")
print(f"Testing set size: {test_set.shape[0]}")

# Define predictor variables and response variable
X_train = train_set.drop(columns=['lettr'])  # Predictor variables (all columns except 'lettr')
y_train = train_set['lettr']  # Response variable

# Add a constant to the predictor variable (intercept term)
X_train_with_intercept = sm.add_constant(X_train)
X_train_with_intercept = X_train_with_intercept.fillna(0)  # Or use column mean
X_train_with_intercept = X_train_with_intercept.replace([np.inf, -np.inf], 0)

Training set size: 14999
Testing set size: 5000


In [195]:
# 4A with Stats Models
try:
    mnlogit_model = sm.MNLogit(y_train, X_train_with_intercept).fit()
    # Display the model summary
    print(mnlogit_model.summary())
except Exception as e:
    print(f"An error occurred while fitting the model: {e}")


KeyboardInterrupt: 

In [ ]:
# 4a - SK learn
# Split data
# Step 3: Preprocessing (Scaling the features)
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_with_intercept)

# Step 4: Train Multinomial Logistic Regression Model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)
model.fit(X_train_scaled, y_train)

coefficients = model.coef_  # Shape: (n_classes, n_features)
intercepts = model.intercept_  # Shape: (n_classes,)

# Classes and shape details
print(f"Classes: {model.classes_}")
print(f"Number of features: {X_train.shape[1]}")
print(f"Number of classes: {len(model.classes_)}")

# Coefficients
print("Coefficients (one row per class):")
print(coefficients)

# Intercepts
print("Intercepts (one per class):")
print(intercepts)


Classes: ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R'
 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z']
Number of features: 16
Number of classes: 26
Coefficients (one row per class):
[[ 0.00000000e+00 -1.39707171e+00  8.15916368e-01  2.31699964e+00
   6.46227541e-01 -1.32534192e+00  2.23421575e+00 -1.18100609e+00
  -1.30997837e+00 -3.11212536e+00 -1.97562111e+00 -9.35449576e-01
   1.92292273e+00 -1.04256938e+00 -8.65919674e-01 -6.92879547e-01
  -1.65182635e+00]
 [ 0.00000000e+00 -1.19181790e-01  1.37934151e-01 -1.30997524e+00
  -1.18557881e+00  1.69781189e+00 -2.99067809e-02 -8.08245838e-01
   9.43519066e-01  1.09780154e+00  1.46633460e+00  7.36096183e-02
  -2.38482458e+00 -4.36412457e-01 -4.51266710e-01  2.11185847e+00
   9.40003098e-01]
 [ 0.00000000e+00  8.64172234e-01  6.42915213e-01 -1.23261974e+00
  -4.99887203e-01 -2.65522428e-01 -6.94275961e-01  1.35883888e+00
   4.16010999e-01  1.32279916e+00  2.99617844e-01  1.43901862e-01
   3.47862153e+00  8.36407638e-01  8.0410